In [ ]:
import streamlit as st
import streamlit.components.v1 as components
import base64
import datetime
from io import BytesIO
# HTML과 JavaScript를 사용하여 녹음 기능 구현
st.title("Voice Recorder")
# HTML과 JavaScript 삽입
html_code = """
    <h3>Click the button to record your voice:</h3>
    <button id="recordButton">Start Recording</button>
    <button id="stopButton" disabled>Stop Recording</button>
    <p id="status">Not recording...</p>
    <audio id="audioPlayback" controls></audio>
    <script>
        let chunks = [];
        let recorder;
        let audioBlob;
        const recordButton = document.getElementById('recordButton');
        const stopButton = document.getElementById('stopButton');
        const status = document.getElementById('status');
        const audioPlayback = document.getElementById('audioPlayback');
        if (navigator.mediaDevices && navigator.mediaDevices.getUserMedia) {
            recordButton.onclick = () => {
                navigator.mediaDevices.getUserMedia({ audio: true }).then(stream => {
                    recorder = new MediaRecorder(stream);
                    recorder.ondataavailable = event => chunks.push(event.data);
                    recorder.onstop = () => {
                        audioBlob = new Blob(chunks, { type: 'audio/wav' });
                        chunks = [];
                        const audioUrl = URL.createObjectURL(audioBlob);
                        audioPlayback.src = audioUrl;
                        status.textContent = 'Recording stopped';
                        stopButton.disabled = true;
                        recordButton.disabled = false;
                        const reader = new FileReader();
                        reader.readAsDataURL(audioBlob);
                        reader.onloadend = function() {
                            const base64data = reader.result.split(',')[1];
                            fetch('/upload', {
                                method: 'POST',
                                headers: {
                                    'Content-Type': 'application/json',
                                },
                                body: JSON.stringify({ audio: base64data }),
                            })
                            .then(response => response.json())
                            .then(data => {
                                status.textContent = 'File saved as ' + data.filename;
                            })
                            .catch((error) => {
                                console.error('Error:', error);
                            });
                        }
                    };
                    recorder.start();
                    status.textContent = 'Recording...';
                    stopButton.disabled = false;
                    recordButton.disabled = true;
                }).catch(err => {
                    console.error('Error accessing media devices.', err);
                    status.textContent = 'Error accessing media devices.';
                });
            };
            stopButton.onclick = () => {
                if (recorder) {
                    recorder.stop();
                }
            };
        } else {
            alert('getUserMedia is not supported in this browser.');
        }
    </script>
"""
# HTML 컴포넌트 삽입
components.html(html_code, height=400)
# Streamlit 앱에서 전송된 오디오 데이터 처리
def save_audio(audio_data):
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"recording_{timestamp}.wav"
    with open(filename, "wb") as f:
        f.write(base64.b64decode(audio_data))
    return filename
# URL 쿼리 파라미터에서 오디오 데이터 추출
def handle_audio_upload():
    if st.experimental_get_query_params():
        audio_data = st.experimental_get_query_params().get('audio', [None])[0]
        if audio_data:
            saved_filename = save_audio(audio_data)
            st.success(f"File saved as {saved_filename}")
            st.audio(saved_filename)
handle_audio_upload()